<a href="https://colab.research.google.com/github/arahogc/Whats_In_A_Bottle/blob/Jess/description_parsing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Initialize Spark
import os

# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
#spark_version = 'spark-2.4.5'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:14 

In [2]:
# Install Spark NLP
! pip install --ignore-installed spark-nlp==2.4.5

! pip install pyspark

# Install nltk
! pip install nltk

     |████████████████████████████████| 112kB 5.4MB/s 
     |████████████████████████████████| 212.3MB 63kB/s 
     |████████████████████████████████| 204kB 38.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=b552a361021eebec74e74714c8d98cfc225adaea6447a00c63a983bf7dea8e63
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [3]:
# Import spark nlp
import sparknlp
spark = sparknlp.start()

In [4]:
# Import dependencies
import pandas as pd 

from pyspark import SparkFiles
from pyspark.sql import SparkSession

from pyspark.ml.feature import Tokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Import the stop words library
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, StopWordsRemover, Normalizer, StringIndexer

In [5]:
# Start Spark session
spark = SparkSession.builder.appName("Whats_In_A_Bottle").getOrCreate()

In [6]:
# Read in data from S3 Buckets
url ="https://whats-in-a-bottle.s3-us-west-1.amazonaws.com/winemag-data_first150k.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("winemag-data_first150k.csv"), sep=",", header=True)
df.show()

+---+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|_c0|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+---+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|  0|     US|This tremendous 1...|   Martha's Vineyard|    96|235.0|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  1|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|110.0|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|  2|     US|Mac Watson honors...|Special Selected ...|    96| 90.0|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley

In [7]:
# Get count of both null and missing values in pyspark
#from pyspark.sql.functions import isnan, when, count, col

#df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

In [8]:
#df = df.drop("region_2")
#df.show()

In [9]:
# Drop rows with null values
#df = df.na.drop()
#df.show(truncate=False)

In [10]:
#Count the region unique values
region_df = df.groupby(df.region_1).count().distinct()
region_df.orderBy(col('count').desc()).show()

+--------------------+-----+
|            region_1|count|
+--------------------+-----+
|                null|25059|
|         Napa Valley| 6205|
|Columbia Valley (WA)| 4975|
|             Mendoza| 3586|
|Russian River Valley| 3571|
|          California| 3460|
|         Paso Robles| 3053|
|   Willamette Valley| 2096|
|               Rioja| 1893|
|             Toscana| 1885|
|       Sonoma County| 1852|
|Brunello di Monta...| 1746|
|             Sicilia| 1701|
|              Alsace| 1574|
|        Sonoma Coast| 1473|
|            Carneros| 1458|
|    Dry Creek Valley| 1398|
|              Barolo| 1397|
|        Finger Lakes| 1372|
|           Champagne| 1369|
+--------------------+-----+
only showing top 20 rows



In [11]:
# Bin the region_1 data?

In [12]:
# Create a function to return the length of a list
def word_list_length(word_list):
  return len(word_list)

In [13]:
# Create a user defined function
count_tokens = udf(word_list_length, IntegerType())

In [14]:
# Tokenize sentences
tokenizer = Tokenizer(inputCol="description", outputCol="words")
tokenizer

Tokenizer_d8aad034bff1

In [15]:
# Create our Tokenizer
tokenizer = Tokenizer(inputCol="description", outputCol="words")

# Transform DataFrame
tokenized_df = tokenizer.transform(df)

# Select the needed column and don't truncate results
tokenized_df = tokenized_df.withColumn("tokens", count_tokens(col("words")))
tokenized_df.show(truncate=False)

+---+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+------+-----+------------------+-------------------------+-----------------+------------------+-----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
# Use the word remover
# Run the remover
remover = StopWordsRemover(inputCol="words",outputCol="filtered")
# Tranform and show data
removed_df = remover.transform(tokenized_df)
removed_df.show(truncate=False)

+---+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+------+-----+------------------+-------------------------+-----------------+------------------+-----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [41]:
# Select the needed column and don't truncate results
filtered_tokens_df = removed_df.withColumn("filtered_tokens", count_tokens(col("filtered")))
filtered_tokens_df.show(truncate=False)

+---+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+------+-----+------------------+-------------------------+-----------------+------------------+-----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [19]:
! pip install nltk

In [42]:
# Run the hashing term frequency
hashing = HashingTF(inputCol="filtered", outputCol="hashedValues", numFeatures=pow(2,18))

# Transform into a DF
hashed_df = hashing.transform(filtered_tokens_df)
hashed_df.show(truncate=False)

+---+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------+------+-----+------------------+-------------------------+-----------------+------------------+-----------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [43]:
# Convert region to number
region_to_num = StringIndexer(inputCol='region_1', outputCol='label')

# Create a run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[region_to_num])

# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(hashed_df)
cleaned = cleaner.transform(hashed_df)
cleaned.show()

+---+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+--------------------+------+--------------------+---------------+--------------------+-----+
|_c0|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|               words|tokens|            filtered|filtered_tokens|        hashedValues|label|
+---+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+--------------------+------+--------------------+---------------+--------------------+-----+
|  0|     US|This tremendous 1...|   Martha's Vineyard|    96|235.0|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|[this, tremendous...|    60|[tremendous, 100%...|        

In [49]:
# Fit the IDF on the data set
idf = IDF(inputCol="hashedValues", outputCol="features")
idfModel = idf.fit(cleaned)
rescaledData = idfModel.transform(cleaned)

# Display the DataFrame
rescaledData.show(truncate=False)

Py4JJavaError: ignored

In [23]:
# Need to run this data through a model (label=region, features=filitered)